# Learn the domain model in PDDL using iLOCM

**interactive-LOCM**
This code combines LOCM1 and LOCM2 algorithms and is last part of the pipeline that I use in my thesis to generate PDDL models from instructional texts.

- Step 0: Preprocess: Lemmatize, Coref resolve, action override rename and replacing empty parameters.
- Step 1: Find classes and make transition graphs.
- Step 2: Get transistion sets from LOCM2 algorithm
- Step 3: Create FSMs
- Step 4: Perform Zero Analysis and add new FSM if necessary.
- Step 5: Create and test hypothesis for state parameters
- Step 6: Create and merge state parameters
- Step 7: Remove parameter flaws
- Step 8: Extract static preconditions
- Step 9: Form action schemas

In [93]:
from collections import defaultdict
import itertools
import os
from tabulate import tabulate
from pprint import pprint
import matplotlib.pyplot as plt
%matplotlib inline
import networkx as nx
import pandas as pd
pd.options.display.max_columns = 100
from IPython.display import display, Markdown
from ipycytoscape import *

## Read input file

In [43]:
import string
def read_file(input_file_name):
    '''
    Read the input data and return list of action sequences.
    Each sequence is a list of action-argumentlist tuples.
    '''
    file = open(input_file_name, 'r')
    sequences = []
    for line in file:
        
        actions = []
        arguments = []
        if line and not line.isspace() and len(line)>1:
            sequence = line.rstrip("\n\r").lstrip("\n\r").lower() 
            action_defs = sequence.split("),")

            for action_def in action_defs:
                action = action_def.split('(')[0].strip(")\n\r").strip()
                argument = action_def.split('(')[1].strip(")\n\r")
                actions.append(action.translate(str.maketrans('', '', string.punctuation)))
                argument_list = argument.split(',')
                argument_list = [x.strip() for x in argument_list]
                #argument_list.insert(0,'zero')
                arguments.append(argument_list)
                
            
            actarg_tuples = zip(actions,arguments)
            sequences.append(list(actarg_tuples))
    return sequences

def print_sequences(sequences):
    for seq in sequences:
        for index,action in enumerate(seq):
            print(str(index) + ": " + str(action))
        print()

In [126]:
# sequences = read_file(input_file_name)
sequences = read_file('./locm_data/driverlog1.txt')
print_sequences(sequences)
domain_name = 'driverlog' #specify domain name to  be used in PDDL here.

0: ('walk', ['driver1', 's2', 'p1-2'])
1: ('walk', ['driver1', 'p1-2', 's1'])
2: ('walk', ['driver1', 's1', 'p1-0'])
3: ('walk', ['driver1', 'p1-0', 's0'])
4: ('boardtruck', ['driver1', 'truck1', 's0'])
5: ('drivetruck', ['truck1', 's0', 's1', 'driver1'])
6: ('disembarktruck', ['driver1', 'truck1', 's1'])

0: ('boardtruck', ['driver1', 'truck3', 's3'])
1: ('unloadtruck', ['package5', 'truck3', 's3'])
2: ('disembarktruck', ['driver1', 'truck3', 's3'])

0: ('unloadtruck', ['package5', 'truck3', 's3'])
1: ('boardtruck', ['driver1', 'truck3', 's3'])
2: ('loadtruck', ['package5', 'truck3', 's3'])

0: ('unloadtruck', ['package5', 'truck3', 's3'])
1: ('loadtruck', ['package5', 'truck3', 's3'])

0: ('loadtruck', ['package5', 'truck3', 's3'])
1: ('disembarktruck', ['driver1', 'truck3', 's3'])

0: ('boardtruck', ['driver1', 'truck3', 's3'])
1: ('disembarktruck', ['driver1', 'truck3', 's3'])
2: ('loadtruck', ['package5', 'truck3', 's3'])
3: ('boardtruck', ['driver1', 'truck3', 's3'])

0: ('walk',

## Step 1.1: Find classes 

In [127]:
transitions = set() # A transition is denoted by action_name + argument position
arguments = set()
actions = set()
for seq in sequences:
    for actarg_tuple in seq:
        actions.add(actarg_tuple[0])
        for j, arg in enumerate(actarg_tuple[1]):
            transitions.add(actarg_tuple[0]+"."+str(j))
            arguments.add(arg)

print("\nActions")
print(actions)
# print("\nTransitions")
# print(transitions)
print("\nArguments/Objects")
print(arguments)


Actions
{'disembarktruck', 'drivetruck', 'unloadtruck', 'boardtruck', 'loadtruck', 'walk'}

Arguments/Objects
{'p1-0', 's4', 'package3', 'p1-2', 'p0-1', 's3', 'p3-4', 'driver1', 'p3-2', 's1', 's0', 'package1', 'truck1', 'package4', 'driver2', 's2', 'truck2', 's5', 'truck3', 'package2', 'package5'}


In [128]:
def get_actarg_dictionary(sequences):
    d = defaultdict(list)
    for seq in sequences:
        for actarg_tuple in seq:
            d[actarg_tuple[0]].append(actarg_tuple[1])
    return d
d = get_actarg_dictionary(sequences)

In [129]:
# class util functions.
def get_classes(d):
    # TODO incorporate word similarity in get classes.
    c = defaultdict(set)
    for k,v in d.items():
        for arg_list in v:
            for i,object in enumerate(arg_list):
                c[k,i].add(object)

    sets = c.values()
    classes = []
    # remove duplicate classes
    for s in sets:
        if s not in classes:
            classes.append(s)

    # now do pairwise intersections of all values. If intersection, combine them; then return the final sets.
    classes_copy = list(classes)
    while True:
        combinations = list(itertools.combinations(classes_copy,2))
        intersections_count = 0
        for combination in combinations:
            if combination[0].intersection(combination[1]):
                intersections_count +=1

                if combination[0] in classes_copy:
                    classes_copy.remove(combination[0])
                if combination[1] in classes_copy:
                    classes_copy.remove(combination[1])
                classes_copy.append(combination[0].union(combination[1]))

        if intersections_count==0:
            # print("no intersections left")
            break

    return classes_copy

# TODO: Can use better approach here. NER might help.
def get_class_names(classes):
    # Name the class to first object found ignoring the digits in it
    class_names = []
    for c in classes:
        for object in c:
#             object = ''.join([i for i in object if not i.isdigit()])
            class_names.append(object)
            break
    return class_names

def get_class_index(arg,classes):
    for class_index, c in enumerate(classes):
        if arg in c:
            return class_index #it is like breaking out of the loop
    print("Error:class index not found") #this statement is only executed if class index is not returned.

In [130]:
classes = get_classes(d) #sorts of object
print("\nSorts/Classes")
print(classes)

class_names = get_class_names(classes)
print("\nExtracted class names")
print(class_names)


Sorts/Classes
[{'driver1', 'driver2'}, {'package3', 'package2', 'package5', 'package1', 'package4'}, {'truck3', 'truck2', 'truck1'}, {'p1-0', 's3', 's4', 'p3-4', 's5', 'p3-2', 's1', 's0', 'p1-2', 'p0-1', 's2'}]

Extracted class names
['driver1', 'package3', 'truck3', 'p1-0']


## User Input 1: Enter Correct Class names
Editing the extracted class names to more readable object classes will make the final PDDL model more readable.

In [131]:
############ (Optional) User Input ############
# Give user an option to change class names.
# class_names[0] = 'rocket'

class_names[0] = 'Driver'
class_names[1] = 'Truck'
class_names[2] = 'Package'
class_names[3] = 'Location'

print("\nRenamed class names")
print(class_names)


Renamed class names
['Driver', 'Truck', 'Package', 'Location']


 **Assumptions of LOCM2**
- Each object of a same class undergoes similar kind of transition.
- Objects of same class in a same action undergo similar kind of transition.

In [132]:
# change transitions to be more meaningful by incorporating class_names.
full_transitions = set()
for seq in sequences:
    for actarg_tuple in seq:
        actions.add(actarg_tuple[0])
        for j, arg in enumerate(actarg_tuple[1]):
            full_transitions.add(actarg_tuple[0]+"."+class_names[get_class_index(arg,classes)]+".#"+str(j))
            arguments.add(arg)

print("\nActions")
print(actions)
print("\nTransitions")
print(full_transitions)
print("\nArguments/Objects")
print(arguments)


Actions
{'disembarktruck', 'drivetruck', 'unloadtruck', 'boardtruck', 'loadtruck', 'walk'}

Transitions
{'loadtruck.Package.#1', 'unloadtruck.Truck.#0', 'disembarktruck.Package.#1', 'loadtruck.Truck.#0', 'drivetruck.Driver.#3', 'drivetruck.Location.#2', 'walk.Driver.#0', 'drivetruck.Package.#0', 'unloadtruck.Package.#1', 'drivetruck.Location.#1', 'boardtruck.Driver.#0', 'disembarktruck.Location.#2', 'walk.Location.#1', 'unloadtruck.Location.#2', 'disembarktruck.Driver.#0', 'walk.Location.#2', 'boardtruck.Package.#1', 'boardtruck.Location.#2', 'loadtruck.Location.#2'}

Arguments/Objects
{'p1-0', 's4', 'package3', 'p1-2', 'p0-1', 's3', 'p3-4', 'driver1', 'p3-2', 's1', 's0', 'package1', 'truck1', 'package4', 'driver2', 's2', 'truck2', 's5', 'truck3', 'package2', 'package5'}


In [133]:
print("\nNumber of Actions: {},\nNumber of unique transitions: {},\nNumber of unique objects (arguments): {},\nNumber of classes/sorts: {}".format(len(actions), len(transitions), len(arguments), len(classes)))


Number of Actions: 6,
Number of unique transitions: 19,
Number of unique objects (arguments): 21,
Number of classes/sorts: 4


## Building Transition graphs

### Utils

In [146]:
def empty_directory(folder):
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            # elif os.path.isdir(file_path): shutil.rmtree(file_path)
        except Exception as e:
            print(e)

def findsubsets(S,m):
    return set(itertools.combinations(S, m))

def print_table(matrix):
    display(tabulate(matrix, headers='keys', tablefmt='html'))
    
def printmd(string):
    display(Markdown(string))

### Save graphs in graphml format (used in cytoscape)

In [143]:
def save(graphs, domain_name):
    adjacency_matrix_list = [] # list of adjacency matrices per class
    
    for index, G in enumerate(graphs):
        nx.write_graphml(G, "output/"+ domain_name + "/" +  class_names[index] + ".graphml")
        df = nx.to_pandas_adjacency(G, nodelist=G.nodes(), dtype=int)
        adjacency_matrix_list.append(df)
    return adjacency_matrix_list

In [144]:
def plot_cytographs(graphs, domain_name):
    cytoscapeobs = []
    for index, G in enumerate(graphs):
        cytoscapeobj = CytoscapeWidget()
        cytoscapeobj.graph.add_graph_from_networkx(G)
        cytoscapeobs.append(cytoscapeobj)
        printmd('## class **'+class_names[index]+'**')
        print("Nodes:{}".format(G.nodes()))
        print("Edges:{}".format(G.edges()))
        cytoscapeobj.set_style([{
                        'width':300,
                        'height':300,
            
                        'selector': 'node',
                        'style': {
                            'label': 'data(id)',
                            'font-family': 'helvetica',
                            'font-size': '8px',
                            'background-color': '#11479e',
                            'height':'10px',
                            'width':'10px',
                            
                            
                            }
    
                        },
                        {
                        'selector': 'node:parent',
                        'css': {
                            'background-opacity': 0.333,
                            'background-color': '#bbb'
                            }
                        },
                        {
                        'selector': '$node > node',
                        'css': {
                            'padding-top': '10px',
                            'padding-left': '10px',
                            'padding-bottom': '10px',
                            'padding-right': '10px',
                            'text-valign': 'top',
                            'text-halign': 'center',
                            'background-color': '#bbb'
                          }
                        },
                       {
                            'selector': 'edge',
                            
                            'style': {
                                'label':'data(weight)',
                                'width': 1,
                                'line-color': '#9dbaea',
                                'target-arrow-shape': 'triangle',
                                'target-arrow-color': '#9dbaea',
                                'arrow-scale': 0.5,
                                'curve-style': 'bezier',
                                'font-family': 'helvetica',
                                'font-size': '8px',
                                'text-valign': 'top',
                                'text-halign':'center'
                            }
                        },
                        ])
        cytoscapeobj.max_zoom = 4.0
        cytoscapeobj.min_zoom = 0.5
        display(cytoscapeobj)
    return cytoscapeobs

#### Build transitions graphs and call save function

In [137]:
def build_and_save_transition_graphs(classes, domain_name, class_names):
    # There should be a graph for each class of objects.
    graphs = []
    # Initialize all graphs empty
    for sort in classes:
        graphs.append(nx.DiGraph())

    consecutive_transition_lists = [] #list of consecutive transitions per object instance per sequence.

    for m, arg in enumerate(arguments):  # for all arguments (objects found in sequences)
        for n, seq in enumerate(sequences):  # for all sequences
            consecutive_transition_list = list()  # consecutive transition list for a sequence and an object (arg)
            for i, actarg_tuple in enumerate(seq):
                for j, arg_prime in enumerate(actarg_tuple[1]):  # for all arguments in actarg tuples
                    if arg == arg_prime:  # if argument matches arg
                        node = actarg_tuple[0] + "." +  str(j)
                        # node = actarg_tuple[0] +  "." + class_names[get_class_index(arg,classes)] + "." +  str(j)  # name the node of graph which represents a transition
                        consecutive_transition_list.append(node)  # add node to the cons_transition for sequence and argument

                        # for each class append the nodes to the graph of that class
                        class_index = get_class_index(arg_prime, classes)  # get index of class to which the object belongs to
                        graphs[class_index].add_node(node)  # add node to the graph of that class

            consecutive_transition_lists.append([n, arg, consecutive_transition_list])

    # print(consecutive_transition_lists)
    # for all consecutive transitions add edges to the appropriate graphs.
    for cons_trans_list in consecutive_transition_lists:
        # print(cons_trans_list)
        seq_no = cons_trans_list[0]  # get sequence number
        arg = cons_trans_list[1]  # get argument
        class_index = get_class_index(arg, classes)  # get index of class
        # add directed edges to graph of that class
        for i in range(0, len(cons_trans_list[2]) - 1):
                if graphs[class_index].has_edge(cons_trans_list[2][i], cons_trans_list[2][i + 1]):
                    graphs[class_index][cons_trans_list[2][i]][cons_trans_list[2][i + 1]]['weight'] += 1
                else:
                    graphs[class_index].add_edge(cons_trans_list[2][i], cons_trans_list[2][i + 1], weight=1)


    
    # make directory if doesn't exist
    dirName = "output/"+ domain_name
    if not os.path.exists(dirName):
        os.makedirs(dirName)
        print("Directory ", dirName, " Created ")
    else:
        print("Directory ", dirName, " already exists")
    empty_directory(dirName)

    # save all the graphs
    adjacency_matrix_list = save(graphs, domain_name) # list of adjacency matrices per class
    
    # plot cytoscape interactive graphs
    cytoscapeobs = plot_cytographs(graphs,domain_name)
    
    return adjacency_matrix_list, graphs, cytoscapeobs

##### Transition Graphs

In [138]:
#### Build weighted directed graphs for transitions.
printmd("## "+ domain_name.upper())
adjacency_matrix_list, graphs, cytoscapeobjs = build_and_save_transition_graphs(classes, domain_name, class_names)

## DRIVERLOG

Directory  output/driverlog  already exists


## class **Driver**

Nodes:['walk.0', 'boardtruck.0', 'drivetruck.3', 'disembarktruck.0']
Edges:[('walk.0', 'walk.0'), ('walk.0', 'boardtruck.0'), ('boardtruck.0', 'drivetruck.3'), ('boardtruck.0', 'disembarktruck.0'), ('drivetruck.3', 'disembarktruck.0'), ('drivetruck.3', 'drivetruck.3'), ('disembarktruck.0', 'boardtruck.0'), ('disembarktruck.0', 'walk.0')]


CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'width': 300, 'height': 300, 'selector': …

...

Nodes:['boardtruck.1', 'drivetruck.0', 'disembarktruck.1', 'loadtruck.1', 'unloadtruck.1']
Edges:[('boardtruck.1', 'drivetruck.0'), ('boardtruck.1', 'loadtruck.1'), ('boardtruck.1', 'unloadtruck.1'), ('boardtruck.1', 'disembarktruck.1'), ('drivetruck.0', 'disembarktruck.1'), ('drivetruck.0', 'unloadtruck.1'), ('drivetruck.0', 'drivetruck.0'), ('drivetruck.0', 'loadtruck.1'), ('disembarktruck.1', 'unloadtruck.1'), ('disembarktruck.1', 'boardtruck.1'), ('disembarktruck.1', 'loadtruck.1'), ('loadtruck.1', 'drivetruck.0'), ('loadtruck.1', 'unloadtruck.1'), ('loadtruck.1', 'disembarktruck.1'), ('loadtruck.1', 'boardtruck.1'), ('unloadtruck.1', 'drivetruck.0'), ('unloadtruck.1', 'disembarktruck.1'), ('unloadtruck.1', 'boardtruck.1'), ('unloadtruck.1', 'loadtruck.1')]


CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'width': 300, 'height': 300, 'selector': …

## class **Location**

Nodes:['walk.2', 'walk.1', 'boardtruck.2', 'loadtruck.2', 'drivetruck.1', 'drivetruck.2', 'disembarktruck.2', 'unloadtruck.2']
Edges:[('walk.2', 'walk.1'), ('walk.2', 'drivetruck.2'), ('walk.2', 'boardtruck.2'), ('walk.1', 'drivetruck.2'), ('walk.1', 'walk.2'), ('boardtruck.2', 'loadtruck.2'), ('boardtruck.2', 'drivetruck.1'), ('boardtruck.2', 'unloadtruck.2'), ('boardtruck.2', 'disembarktruck.2'), ('loadtruck.2', 'drivetruck.1'), ('loadtruck.2', 'disembarktruck.2'), ('loadtruck.2', 'boardtruck.2'), ('loadtruck.2', 'unloadtruck.2'), ('drivetruck.1', 'drivetruck.2'), ('drivetruck.1', 'walk.2'), ('drivetruck.1', 'boardtruck.2'), ('drivetruck.2', 'drivetruck.1'), ('drivetruck.2', 'loadtruck.2'), ('drivetruck.2', 'disembarktruck.2'), ('drivetruck.2', 'unloadtruck.2'), ('disembarktruck.2', 'boardtruck.2'), ('disembarktruck.2', 'loadtruck.2'), ('disembarktruck.2', 'walk.1'), ('disembarktruck.2', 'unloadtruck.2'), ('disembarktruck.2', 'drivetruck.2'), ('unloadtruck.2', 'disembarktruck.2'), ('

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'width': 300, 'height': 300, 'selector': …

## User Input 2: Edit transition graphs
For meaningful LOCM models, here one can edit the transition graphs to make them accurate. However, in the paper we don't do that in order to estimate what kind of models are learned automatically from natural language data.

Option 1. **You can add or delete nodes/edges in transition graphs by following methods like add_node, delete_edges shown in the following library.**
https://github.com/QuantStack/ipycytoscape/blob/master/ipycytoscape/cytoscape.py

Option 2. **Alternatively you can use the saved .graphml file. Open it up in Cytoscape, edit it within the GUI and load that graph into the graphs list.**

## Step 2: Get Transition Sets from LOCM2

**Algorithm**: LOCM2

**Input** : 
- T_all = set of observed transitions for a sort/class
- H : Set of holes - each hole is a set of two transitions.
- P : Set of pairs <t1,t2> i.e. consecutive transitions.
- E : Set of example sequences of actions.

**Output**:
- S : Set of transition sets.
### Finding holes
Holes are transitions that LOCM1 will assume to be true due to the flaw of overgeneralizing

In [139]:
def get_adjacency_matrix_with_holes(adjacency_matrix_list):
    adjacency_matrix_list_with_holes = []
    for index,adjacency_matrix in enumerate(adjacency_matrix_list):
        # print("\n ROWS ===========")
        df = adjacency_matrix.copy()
        df1 = adjacency_matrix.copy()

        # for particular adjacency matrix's copy, loop over all pairs of rows
        for i in range(df.shape[0] - 1):
            for j in range(i+1, df.shape[0]):
                idx1, idx2 = i, j
                row1, row2 = df.iloc[idx1,:], df.iloc[idx2, :] #we have now all pairs of rows

                common_values_flag = False #for each two rows we have a common_values_flag

                # if there is a common value between two rows, turn common value flag to true
                for col in range(row1.shape[0]):
                    if row1.iloc[col] > 0 and row2.iloc[col] > 0:
                        common_values_flag = True
                        break

                # now if two rows have common values, we need to check for holes.
                if common_values_flag:
                    for col in range(row1.shape[0]):
                        if row1.iloc[col] > 0 and row2.iloc[col] == 0:
                            df1.iloc[idx2,col] = 'hole'
                        elif row1.iloc[col] == 0 and row2.iloc[col] > 0:
                            df1.iloc[idx1, col] = 'hole'

        adjacency_matrix_list_with_holes.append(df1)
    return adjacency_matrix_list_with_holes

In [149]:
adjacency_matrix_list_with_holes = get_adjacency_matrix_with_holes(adjacency_matrix_list)

# Printing FSM matrices with and without holes
for index,adjacency_matrix in enumerate(adjacency_matrix_list):
    printmd("\n#### " + class_names[index] )
    print_table(adjacency_matrix)

    printmd("\n#### HOLES: " + class_names[index])
    print_table(adjacency_matrix_list_with_holes[index])


#### Driver

,walk.0,boardtruck.0,drivetruck.3,disembarktruck.0
walk.0,10,4,0,0
boardtruck.0,0,0,11,2
drivetruck.3,0,0,20,11
disembarktruck.0,2,6,0,0



#### HOLES: Driver

,walk.0,boardtruck.0,drivetruck.3,disembarktruck.0
walk.0,10,4,0,0
boardtruck.0,0,0,11,2
drivetruck.3,0,0,20,11
disembarktruck.0,2,6,0,0



#### Truck

...

,boardtruck.1,drivetruck.0,disembarktruck.1,loadtruck.1,unloadtruck.1
boardtruck.1,hole,7,1,5,1
drivetruck.0,hole,10,9,5,7
disembarktruck.1,5,hole,hole,1,1
loadtruck.1,1,8,1,hole,1
unloadtruck.1,1,6,3,1,hole



#### Location

,walk.2,walk.1,boardtruck.2,loadtruck.2,drivetruck.1,drivetruck.2,disembarktruck.2,unloadtruck.2
walk.2,0,10,4,0,0,1,0,0
walk.1,1,0,0,0,0,5,0,0
boardtruck.2,0,0,0,5,7,0,1,1
loadtruck.2,0,0,1,0,8,0,1,1
drivetruck.1,2,0,1,0,0,20,0,0
drivetruck.2,0,0,0,5,10,0,9,7
disembarktruck.2,0,2,4,1,0,1,0,1
unloadtruck.2,0,0,1,1,6,0,3,0



#### HOLES: Location

,walk.2,walk.1,boardtruck.2,loadtruck.2,drivetruck.1,drivetruck.2,disembarktruck.2,unloadtruck.2
walk.2,hole,10,4,hole,hole,1,hole,hole
walk.1,1,hole,hole,hole,0,5,0,hole
boardtruck.2,0,hole,hole,5,7,hole,1,1
loadtruck.2,hole,hole,1,hole,8,hole,1,1
drivetruck.1,2,hole,1,hole,hole,20,hole,hole
drivetruck.2,0,hole,hole,5,10,hole,9,7
disembarktruck.2,hole,2,4,1,hole,1,hole,1
unloadtruck.2,hole,hole,1,1,6,hole,3,hole


In [160]:
# Create list of set of holes per class (H)
holes_per_class = []

for index,df in enumerate(adjacency_matrix_list_with_holes):
    holes = set()
    for i in range(df.shape[0]):
        for j in range(df.shape[1]):
            if df.iloc[i,j] == 'hole':
                holes.add(frozenset({df.index[i] , df.columns[j]}))
    holes_per_class.append(holes)
for i, hole in enumerate(holes_per_class):
    print("#holes in class " + class_names[i]+":" + str(len(hole)))
#     for h in hole:
#         print(list(h))

#holes in class Driver:0
#holes in class Truck:0
#holes in class Package:6
#holes in class Location:23


In [172]:
# List of transitions per class (T_all). It is just a set of transitions that occur for a class.
transitions_per_class = []
for index, df in enumerate(adjacency_matrix_list_with_holes):
    transitions_per_class.append(df.columns.values)
for i, transition in enumerate(transitions_per_class):
    print('{}:{}'.format(class_names[i], transition))

Driver:['walk.0' 'boardtruck.0' 'drivetruck.3' 'disembarktruck.0']
Truck:['loadtruck.0' 'unloadtruck.0']
Package:['boardtruck.1' 'drivetruck.0' 'disembarktruck.1' 'loadtruck.1'
 'unloadtruck.1']
Location:['walk.2' 'walk.1' 'boardtruck.2' 'loadtruck.2' 'drivetruck.1'
 'drivetruck.2' 'disembarktruck.2' 'unloadtruck.2']


In [173]:
def get_consecutive_transitions_per_class(adjacency_matrix_list_with_holes):
    consecutive_transitions_per_class = []
    for index, df in enumerate(adjacency_matrix_list_with_holes):
        consecutive_transitions = set()  # for a class
        for i in range(df.shape[0]):
            for j in range(df.shape[1]):
                if df.iloc[i, j] != 'hole':
                    if df.iloc[i, j] > 0:
#                         print("(" + df.index[i] + "," + df.columns[j] + ")")
                        consecutive_transitions.add((df.index[i], df.columns[j]))
        consecutive_transitions_per_class.append(consecutive_transitions)
    return consecutive_transitions_per_class

In [174]:
#  Create list of consecutive transitions per class (P). If value is not null, ordered pair i,j would be consecutive transitions per class
consecutive_transitions_per_class = get_consecutive_transitions_per_class(adjacency_matrix_list_with_holes)
for i, transition in enumerate(consecutive_transitions_per_class):
    printmd("#### "+class_names[i]+":")
    for x in list(transition):
        print(x)
#     print('{}:{}'.format(class_names[i], transition))
    print()

#### Driver:

('walk.0', 'walk.0')
('boardtruck.0', 'drivetruck.3')
('drivetruck.3', 'disembarktruck.0')
('disembarktruck.0', 'walk.0')
('drivetruck.3', 'drivetruck.3')
('walk.0', 'boardtruck.0')
('disembarktruck.0', 'boardtruck.0')
('boardtruck.0', 'disembarktruck.0')



#### Truck:

('loadtruck.0', 'unloadtruck.0')
('unloadtruck.0', 'loadtruck.0')



#### Package:

('drivetruck.0', 'disembarktruck.1')
('boardtruck.1', 'disembarktruck.1')
('boardtruck.1', 'drivetruck.0')
('unloadtruck.1', 'drivetruck.0')
('drivetruck.0', 'drivetruck.0')
('disembarktruck.1', 'unloadtruck.1')
('drivetruck.0', 'unloadtruck.1')
('disembarktruck.1', 'loadtruck.1')
('boardtruck.1', 'unloadtruck.1')
('loadtruck.1', 'boardtruck.1')
('disembarktruck.1', 'boardtruck.1')
('unloadtruck.1', 'disembarktruck.1')
('unloadtruck.1', 'loadtruck.1')
('loadtruck.1', 'drivetruck.0')
('boardtruck.1', 'loadtruck.1')
('unloadtruck.1', 'boardtruck.1')
('loadtruck.1', 'disembarktruck.1')
('drivetruck.0', 'loadtruck.1')
('loadtruck.1', 'unloadtruck.1')



#### Location:

('walk.2', 'boardtruck.2')
('boardtruck.2', 'disembarktruck.2')
('boardtruck.2', 'drivetruck.1')
('boardtruck.2', 'loadtruck.2')
('loadtruck.2', 'disembarktruck.2')
('unloadtruck.2', 'disembarktruck.2')
('drivetruck.1', 'boardtruck.2')
('unloadtruck.2', 'drivetruck.1')
('disembarktruck.2', 'walk.1')
('walk.1', 'drivetruck.2')
('drivetruck.1', 'drivetruck.2')
('drivetruck.2', 'drivetruck.1')
('drivetruck.1', 'walk.2')
('disembarktruck.2', 'boardtruck.2')
('drivetruck.2', 'disembarktruck.2')
('walk.2', 'drivetruck.2')
('disembarktruck.2', 'drivetruck.2')
('disembarktruck.2', 'loadtruck.2')
('drivetruck.2', 'unloadtruck.2')
('walk.1', 'walk.2')
('unloadtruck.2', 'loadtruck.2')
('walk.2', 'walk.1')
('loadtruck.2', 'unloadtruck.2')
('disembarktruck.2', 'unloadtruck.2')
('boardtruck.2', 'unloadtruck.2')
('unloadtruck.2', 'boardtruck.2')
('loadtruck.2', 'boardtruck.2')
('drivetruck.2', 'loadtruck.2')
('loadtruck.2', 'drivetruck.1')



### Check Well Formed

In [ ]:
def check_well_formed(subset_df):
    # got the adjacency matrix subset
    df = subset_df.copy()

    # for particular adjacency matrix's copy, loop over all pairs of rows
    for i in range(df.shape[0] - 1):
        for j in range(i + 1, df.shape[0]):
            idx1, idx2 = i, j
            row1, row2 = df.iloc[idx1, :], df.iloc[idx2, :]  # we have now all pairs of rows

            common_values_flag = False  # for each two rows we have a common_values_flag

            # if there is a common value between two rows, turn common value flag to true
            for col in range(row1.shape[0]):
                if row1.iloc[col] > 0 and row2.iloc[col] > 0:
                    common_values_flag = True
                    break

            # now if two rows have common values, we need to check for holes.
            if common_values_flag:
                for col in range(row1.shape[0]):
                    if row1.iloc[col] > 0 and row2.iloc[col] == 0:
                        return False
                    elif row1.iloc[col] == 0 and row2.iloc[col] > 0:
                        return False
    return True

### Check Valid Transitions

In [ ]:
def check_valid(subset_df,consecutive_transitions_per_class):

    
    # Reasoning: If we check against all consecutive transitions per class, 
    # we essentially check against all example sequences.
    # check the candidate set which is well-formed (subset df against all consecutive transitions)

    # got the adjacency matrix subset
    df = subset_df.copy()

    # for particular adjacency matrix's copy, loop over all pairs of rows
    for i in range(df.shape[0]):
        for j in range(df.shape[0]):
            if df.iloc[i,j] > 0:
                valid_val_flag = False
                ordered_pair = (df.index[i], df.columns[j])
                for ct_list in consecutive_transitions_per_class:
                    for ct in ct_list:
                        if ordered_pair == ct:
                            valid_val_flag=True
                # if after all iteration ordered pair is not found, mark the subset as invalid.
                if not valid_val_flag:
                    return False
    return True

In [192]:
def check_validity_in_E(subset_df,sequences):

    # got the adjacency matrix subset
    df = subset_df.copy()

    # for particular adjacency matrix's copy, loop over all pairs of rows
    for i in range(df.shape[0]):
        for j in range(df.shape[0]):
            if df.iloc[i,j] > 0:
                valid_val_flag = False
                ordered_pair = (df.index[i], df.columns[j])
                
                for sequence in sequences:
                    for actarg_tuple in sequence:
                        print(actarg_tuple)
#                 for ct_list in consecutive_transitions_per_class:
#                     for ct in ct_list:
#                         if ordered_pair == ct:
#                             valid_val_flag=True
                # if after all iteration ordered pair is not found, mark the subset as invalid.
                if not valid_val_flag:
                    return False
    return True

### LOCM2 transition sets

In [220]:
def locm2_get_transition_sets_per_class(holes_per_class, transitions_per_class, consecutive_transitions_per_class):
    """LOCM 2 Algorithm in the original LOCM2 paper"""
    transition_sets_per_class = []

    # for each hole in a class of objects.
    for index, holes in enumerate(holes_per_class):
        class_name = class_names[index]
        printmd("### "+  class_name)
        transition_set_list = [] #transition_sets_of_a_class, # intially its empty
        
        if len(holes)==0:
            print("no holes")
        
        if len(holes) > 0: # if there are any holes for a class
            print(str(len(holes)) + " holes")
            for ind, hole in enumerate(holes):
               

                printmd("#### Hole " + str(ind + 1) + ": " + str(set(hole)))
                is_hole_already_covered_flag = False
                if len(transition_set_list)>0:
                    for s_prime in transition_set_list:
                        if hole.issubset(s_prime):
                            printmd("Hole "+ str(hole) + " is already covered.")
                            is_hole_already_covered_flag = True
                            break
                     
                
                # discover a set which includes hole and is well-formed and valid against test data.
                # if hole is not covered, do BFS with sets of increasing sizes starting with s=hole
                if not is_hole_already_covered_flag: 
                    s = hole.copy()
                    candidate_sets = []
                    # all subsets of T_all starting from hole's len +1
                    for i in range(len(s)+1,len(transitions_per_class[index])): 
                        subsets = findsubsets(transitions_per_class[index],i)

                        # append the subsets which are subset of
                        for candidate_set in subsets:
                            if s.issubset(candidate_set):
                                candidate_sets.append(set(candidate_set))
                        
                        
                        for candidate_set in candidate_sets:
                            print(candidate_set)

                        printmd("Checking candidate set *" + str(candidate_set) + "* of class **" + class_name + "** for well formedness and Validity")
                        for candidate_set in candidate_sets:
                            subset_df = adjacency_matrix_list[index].loc[list(candidate_set),list(candidate_set)]
                            print_table(subset_df)

                            # checking for well-formedness
                            well_formed_flag = check_well_formed(subset_df)
                            if not well_formed_flag:
                                print("This subset is NOT well-formed")
                                pass

                            # if well-formed validate across the data to remove inappropriate dead-ends
                            # additional check
                            valid_against_data_flag = False
                            if well_formed_flag:
#                                 print_table(subset_df)
                                print("This subset is well-formed.")

                                # validate against all consecutive transitions per class (P)
                                # This checks all sequences consecutive transitions. So, it is validating against (E)
                                valid_against_data_flag = check_valid(subset_df, consecutive_transitions_per_class)
                                if not valid_against_data_flag:
                                    print("This subset is well-formed and invalid against example data")

                            if valid_against_data_flag:
                                print("This subset is valid.")
                                print("Adding this subset " + str(candidate_set) +" to the locm2 transition set.")
                                
                                if candidate_set not in transition_set_list: # do not allow copies.
                                    transition_set_list.append(candidate_set)
                                break
                        print("Hole that is covered now:")
                        print(list(s))



            # print(transition_set_list)
        #step -7 : remove redundant sets
        ts_copy = transition_set_list.copy()
        for i in range(len(ts_copy)):
            for j in range(len(ts_copy)):
                if ts_copy[i] < ts_copy[j]: #if subset
                    if ts_copy[i] in transition_set_list:
                        transition_set_list.remove(ts_copy[i])
                elif ts_copy[i] > ts_copy[j]:
                    if ts_copy[j] in transition_set_list:
                        transition_set_list.remove(ts_copy[j])
#         print("\nRemoved redundancy transition set list")
#         print(transition_set_list)

        #step-8: include all-transitions machine, even if it is not well-formed.
        transition_set_list.append(set(transitions_per_class[index]))
#         print("\nFinal transition set list")
#         print(class_name)
#         print(transition_set_list)


        transition_sets_per_class.append(transition_set_list)
       

    return transition_sets_per_class


############    LOCM2 #################
####    Input ready for LOCM2, Starting LOCM2 algorithm now
####    Step 8:  selecting transition sets (TS) [Main LOCM2 Algorithm]
printmd("### Getting transitions sets for each class using LOCM2")
transition_sets_per_class = locm2_get_transition_sets_per_class(holes_per_class, transitions_per_class, consecutive_transitions_per_class)

### Getting transitions sets for each class using LOCM2

### Driver

no holes


### Truck

no holes


...

This subset is well-formed.
This subset is valid.
Adding this subset {'boardtruck.2', 'drivetruck.2', 'drivetruck.1'} to the locm2 transition set.
Hole that is covered now:
['boardtruck.2', 'drivetruck.2']


#### Hole 22: {'drivetruck.1', 'walk.1'}

Hole frozenset({'drivetruck.1', 'walk.1'}) is already covered.

#### Hole 23: {'drivetruck.2', 'loadtruck.2'}

Hole frozenset({'drivetruck.2', 'loadtruck.2'}) is already covered.

## Step 3: Algorithm For Induction of State Machines

- Input: Action training sequence of length N
- Output: Transition Set TS, Object states OS.

We already have transition set TS per class.

In [36]:
domain_name='driverlog'

In [37]:
print("Step 3: Induction of Finite State Machines")
state_machines_overall_list = [] # list of all state machines
state_dict_overall = [] #list of state dict per class

for index, ts in enumerate(transition_sets_per_class):
    state_machines_per_class = [] # state machines for each class.
    state_dict_per_class = []
    print(class_names[index])
    # print(ts)
    num_fsms = len(ts)
    print("Number of FSMS:" + str(num_fsms))


    #### Add state identifiers
    states_per_transition_set = []
    for transition_set in ts:
        states = set()
        for transition in transition_set:
            states.add(frozenset({"start(" + transition + ")"}))
            states.add(frozenset({"end(" + transition + ")"}))
        states_per_transition_set.append(states)

    # print(states_per_transition_set)

    #### For each pair of consecutive transitions T1, T2 in TS: Unify states end(T1) and start(T2) in set OS
    for fsm_no, transition_set in enumerate(ts):
        transition_df = adjacency_matrix_list[index].loc[list(transition_set), list(transition_set)] #uses transition matrix without holes

        consecutive_transitions_state_machines_per_class = set()  # find consecutive transitions for a state machine in a class.
        for i in range(transition_df.shape[0]):
            for j in range(transition_df.shape[1]):
                if transition_df.iloc[i, j] != 'hole':
                    if transition_df.iloc[i, j] > 0:
                        consecutive_transitions_state_machines_per_class.add((transition_df.index[i], transition_df.columns[j]))

        # for every consecutive transition and for every state of a class, check if that state matches end(t1) or start(t2)
        for ct in consecutive_transitions_state_machines_per_class:
            s1, s2, s3 = -1, -1, -1
            for s in states_per_transition_set[fsm_no]:
                if "end("+ct[0]+")" in s:
                    s1 = s
                if "start("+ct[1]+")" in s:
                    s2 = s
                if s1 != -1 and s2 != - 1: #if they do, combine them.
                    s3 = s1.union(s2) # union

            if s1 != -1 and s2 != -1 and s3 != -1: #for every ct, if we have combined state, we update states_per_transition_set
                if s1 in states_per_transition_set[fsm_no]:
                    states_per_transition_set[fsm_no].remove(s1)
                if s2 in states_per_transition_set[fsm_no]:
                    states_per_transition_set[fsm_no].remove(s2)
                states_per_transition_set[fsm_no].add(s3)

        ## build a state machine now.
        fsm_graph = nx.DiGraph()

        # TODO: consider making a state dictionary for pretty print of frozen set
        state_dict_per_class.insert(fsm_no,{})
        for i, state in enumerate(states_per_transition_set[fsm_no]):
            state_dict_per_class[fsm_no]["state"+str(i)] = state
            fsm_graph.add_node("state"+str(i))

        # transition_df is defined above. Add edges from transitions.
        # print_table(transition_df)
        for i in range(transition_df.shape[0]):
            for j in range(transition_df.shape[1]):
                if transition_df.iloc[i, j] != 'hole':
                    if transition_df.iloc[i, j] > 0:
                        for node in fsm_graph.nodes():
                            starting_node, ending_node = -1,-1
                            if "end("+transition_df.index[i]+")" in state_dict_per_class[fsm_no][node]:
                                starting_node = node
                            if "start("+transition_df.columns[j]+")" in state_dict_per_class[fsm_no][node]:
                                ending_node = node
                            if starting_node != -1 and ending_node != -1:
                                if fsm_graph.has_edge(starting_node, ending_node):
                                    fsm_graph[starting_node][ending_node]['weight'] += transition_df.iloc[i, j]
                                else:
                                    fsm_graph.add_edge(starting_node, ending_node, weight=transition_df.iloc[i, j], name=""+transition_df.index[i])

        df = nx.to_pandas_adjacency(fsm_graph, nodelist=fsm_graph.nodes(), dtype=int)
        
        nx.write_graphml(fsm_graph, "output/" + domain_name + "/" + class_names[index] + "_stateFSM_" + str(fsm_no+1)+ ".graphml")
        state_machines_per_class.append(df)

    state_machines_overall_list.append(state_machines_per_class)
    state_dict_overall.append(state_dict_per_class)

Step 3: Induction of Finite State Machines
Driver
Number of FSMS:1
Truck
Number of FSMS:3
Package
Number of FSMS:1
Location
Number of FSMS:4


In [38]:
# pretty print state dictionary.
def print_state_dictionary(state_dict_overall):
    for i,state_dict_per_class in enumerate(state_dict_overall):
#         print(class_names[i])
        for j,state_dict_per_fsm in enumerate(state_dict_per_class):
#             print()
            for k,v in state_dict_per_fsm.items():
                print(class_names[i]+"_fsm"+str(j)+"_"+ k + ":"+str(list(v)))
#                 print(list(v))
#                 print()
                
# print_state_dictionary(state_dict_overall)

## Step 5: Induction of parameterized state machines
Create and test hypothesis for state parameters

In [39]:
## Step 5 Input: action sequence Seq, Transition set TS, Object set Obs
## Output: HS retained hypotheses for state parameters
## 5.1 Form hypotheses from state machines
print("*****************")
print("Step 5: Induction of Parameterised Finite State Machines")
HS_list = []
for index, fsms_per_class in enumerate(state_machines_overall_list):
    class_name = class_names[index]
    print("CLASS:"+ class_name)
    for fsm_no, fsm in enumerate(fsms_per_class):
        print("FSM:" + str(fsm_no + 1))
        print_table(fsm)

    print("\nTransition set of this class:")
    print(transition_sets_per_class[index])

    # Hypothesis set per class.
    HS_per_class = []
    for fsm_no, transition_set in enumerate(transition_sets_per_class[index]):
        transition_df = adjacency_matrix_list[index].loc[list(transition_set), list(transition_set)]
        consecutive_transitions_state_machines_per_class = set()  # find consecutive transitions for a state machine in a class.
        for i in range(transition_df.shape[0]):
            for j in range(transition_df.shape[1]):
                if transition_df.iloc[i, j] != 'hole':
                    if transition_df.iloc[i, j] > 0:
                        consecutive_transitions_state_machines_per_class.add((transition_df.index[i], transition_df.columns[j]))

        # Step 5.1 for each pair <B.k and C.l> of consecutive transitions in transition set of a state machine.
        # store hypothesis in Hypothesis set
        HS = set()
        for ct in consecutive_transitions_state_machines_per_class:
            B = ct[0].split('.')[0] # action name of T1
            k = int(ct[0].split('.')[1]) # argument index of T1

            C = ct[1].split('.')[0] # action name of T2
            l = int(ct[1].split('.')[1]) # argument index of T2

            # When both actions B and C contain another argument of the same sort G' in position k' and l' respectively, we hypothesise that there may be a relation between sorts G and G'.
            for seq in sequences:
                for actarg_tuple in seq:
                    arglist1 = []
                    arglist2 = []
                    if actarg_tuple[0] == B: #if action name is same as B
                        arglist1 = actarg_tuple[1].copy()
                        arglist1.remove(actarg_tuple[1][k]) # remove k from arglist
                        for actarg_tuple_prime in seq: #loop through seq again.
                            if actarg_tuple_prime[0] == C:
                                arglist2 = actarg_tuple_prime[1].copy()
                                arglist2.remove(actarg_tuple_prime[1][l]) # remove l from arglist

                        # for arg lists of actions B and C, if class is same add a hypothesis set.
                        for i in range(len(arglist1)):
                            for j in range(len(arglist2)):
                                class1 = get_class_index(arglist1[i], classes)
                                class2 = get_class_index(arglist2[j], classes)
                                if class1 == class2:
                                    HS.add((frozenset({"end("+B+"."+ str(k)+")", "start("+C+"."+str(l)+")"}),B,k,i,C,l,j,index,class1))

        ####### Step 5.2 Test Hypothesis against example sequences!!
        # Check hypothesis against sequences.
        ## It performs an inductive process such that the hypotheses can be either refuted or retained according to the example sequence, but it can never be definitely confirmed.
        ## Requires more data than usual.
        HS_copy = HS.copy()
        for hs in HS:
            # for every consecutive transision for a state machine per class.
            for ct in consecutive_transitions_state_machines_per_class:
                A_p = ct[0].split('.')[0]
                m = int(ct[0].split('.')[1])

                A_q = ct[1].split('.')[0]
                n = int(ct[1].split('.')[1])

                if A_p == hs[1] and m == hs[2] and A_q == hs[4] and n == hs[5]:
                    k_prime = hs[3]
                    l_prime = hs[6]
                    for seq in sequences:
                        for actarg_tuple in seq:
                            arglist1 = []
                            arglist2 = []
                            if actarg_tuple[0] == A_p:
                                arglist1 = actarg_tuple[1].copy()
                                arglist1.remove(actarg_tuple[1][m])  # remove k from arglist
                                for actarg_tuple_prime in seq:
                                    if actarg_tuple_prime[0] == A_q:
                                        arglist2 = actarg_tuple_prime[1].copy()
                                        arglist2.remove(actarg_tuple_prime[1][n])  # remove l from arglist

#                                 class1, class2 = -1,-1
#                                 if k_prime < len(arglist1) and l_prime < len(arglist2):
#                                     class1 = get_class_index(arglist1[k_prime], classes)
#                                     class2 = get_class_index(arglist2[l_prime], classes)

#                                 # Refute the hypothesis if classes are not same at the location specified by hypothesis.
#                                 if class1 != -1 and class2!=-1 and class1 != class2:
#                                     if hs in HS_copy:
#                                         HS_copy.remove(hs)
                                object1, object2 = None, None
                                if k_prime < len(arglist1) and l_prime < len(arglist2):
                                    object1 = arglist1[k_prime]
                                    object2 = arglist2[l_prime]
                                
                                if object1 != None and object2 !=None and object1 !=object2:
                                    if hs in HS_copy:
                                        HS_copy.remove(hs)
                        
                                 



        HS_per_class.append(HS_copy)
    HS_list.append(HS_per_class)

*****************
Step 5: Induction of Parameterised Finite State Machines
CLASS:Driver
FSM:1
|        |   state0 |   state1 |
|--------|----------|----------|
| state0 |       19 |        0 |
| state1 |        0 |        3 |

Transition set of this class:
[{'boardtruck.0', 'disembarktruck.0', 'drivetruck.3'}]
CLASS:Truck
FSM:1
|        |   state0 |   state1 |   state2 |   state3 |   state4 |
|--------|----------|----------|----------|----------|----------|
| state0 |        0 |        0 |        0 |        0 |        0 |
| state1 |        0 |        0 |        0 |        0 |        0 |
| state2 |        0 |        0 |        0 |        0 |        0 |
| state3 |        0 |        0 |        0 |        1 |        0 |
| state4 |        0 |        0 |        0 |        0 |        0 |
FSM:2
|        |   state0 |   state1 |   state2 |   state3 |
|--------|----------|----------|----------|----------|
| state0 |        0 |        0 |        0 |        0 |
| state1 |        0 |        0 |     

In [42]:
# printing hypothesis
# print("\n****** HYPOTHESIS SET*********")
# for HS_per_class in HS_list:
#     for HS_per_fsm in HS_per_class:
#         for h in HS_per_fsm:
#             print(h)

## Step 6: Creation and merging of state parameters

In [192]:
print("Step 6: creating and merging state params")
param_bindings_list_overall = []
for classindex, HS_per_class in enumerate(HS_list):
    param_bind_per_class = []
    for HS_per_fsm in HS_per_class:
        param_binding_list = []
        for index,h in enumerate(HS_per_fsm):
            param_binding_list.append((h,"v"+str(index)))

        for i in range(len(param_binding_list)-1):
            for j in range(i+1, len(param_binding_list)):
                h_1 = param_binding_list[i][0]
                h_2 = param_binding_list[j][0]

                if ((h_1[0] == h_2[0] and h_1[1] == h_2[1] and h_1[2] == h_2[2] and h_1[3] == h_2[3]) or (h_1[0] == h_2[0] and h_1[4] == h_2[4] and h_1[5] == h_2[5] and h_1[6] == h_2[6])):
                    new_tuple = (param_binding_list[j][0], param_binding_list[i][1])
                    param_binding_list.remove((param_binding_list[j][0], param_binding_list[j][1]))
                    param_binding_list.insert(j,new_tuple)
        param_bind_per_class.append(param_binding_list)
#         print(class_names[classindex])
#         for pb in param_binding_list:
#             print(pb)
#         print()
    param_bindings_list_overall.append(param_bind_per_class)

Step 6: creating and merging state params


## Step 7: Remove Parameter Flaws

In [193]:
########### Step 5: Removing parameter flaws
# A parameter P associated with an FSM state S is said to be flawed if there exists a transition into S, which does not supply P with a value.
# This may occur when there exists a transition B.k where end(B.k)=S, but there exists no h containing end(B.k)

para_bind_overall_fault_removed = []
for class_index, para_bind_per_class in enumerate(param_bindings_list_overall):
#     print(class_names[class_index])
    para_bind_per_class_fault_removed = []

    # print(state_machines_overall_list[class_index][fsm_index].index.values)
    for fsm_index, transition_set in enumerate(transition_sets_per_class[class_index]):
        transition_df = adjacency_matrix_list[class_index].loc[list(transition_set), list(transition_set)]
        consecutive_transitions_state_machines_per_class = set()  # find consecutive transitions for a state machine in a class.
        for i in range(transition_df.shape[0]):
            for j in range(transition_df.shape[1]):
                if transition_df.iloc[i, j] != 'hole':
                    if transition_df.iloc[i, j] > 0:
                        consecutive_transitions_state_machines_per_class.add(
                            (transition_df.index[i], transition_df.columns[j]))

        # initialize h_exists with false
        h_exists = []
        for param_index, param_bind in enumerate(para_bind_per_class[fsm_index]):
            h_exists.append(False)

        for ct in consecutive_transitions_state_machines_per_class:
            for state in state_machines_overall_list[class_index][fsm_index].index.values:
                if {"end("+ ct[0] + ")"} <= state_dict_overall[class_index][fsm_index][state]:
                    current_state = state_dict_overall[class_index][fsm_index][state]

                    # for every parameter binding which contains subset of current_state, if B and k are there, hypothesis exists
                    for param_index,param_bind in enumerate(para_bind_per_class[fsm_index]):
                        if param_bind[0][0] <= current_state: #subset of current_state of FSM
                            # print(param_bind[0][1])
                            # print(param_bind[0][2])
                            # print(ct[0].split('.')[0])
                            # print(ct[0].split('.')[1])
                            # print()
                            if param_bind[0][1] == ct[0].split('.')[0]:
                                if param_bind[0][2] == int(ct[0].split('.')[1]): #TODO: Do we need to check other things here
                                    h_exists[param_index] = True

        param_bind_per_fsm_copy = para_bind_per_class[fsm_index].copy()
        for param_index, param_bind in enumerate(para_bind_per_class[fsm_index]):
            # if h_exists[param_index]:
            #     print(param_bind[1])
            if not h_exists[param_index]:
                param_bind_per_fsm_copy.remove(param_bind)

        para_bind_per_class_fault_removed.append(param_bind_per_fsm_copy)
    para_bind_overall_fault_removed.append(para_bind_per_class_fault_removed)

In [194]:
print("Fault Removed Parameter Bindings")
# for class_index, para_bind_per_class in enumerate(para_bind_overall_fault_removed):
#     print(class_names[class_index])
#     for fsm_no, para_bind_per_fsm in enumerate(para_bind_per_class):
#         print("Fsm_no:" + str(fsm_no))
#         for p in para_bind_per_fsm:
#             print(p)
#         print()

Fault Removed Parameter Bindings


## Step 9:  Formation of PDDL Schema

In [195]:
# get action schema
print(";;********************Learned PDDL domain******************")
output_file = "output/"+ domain_name + "/" +  domain_name + ".pddl"
write_file = open(output_file, 'w')
write_line = "(define"
write_line += "  (domain "+ domain_name+")\n"
write_line += "  (:requirements :typing)\n"
write_line += "  (:types"
for class_name in class_names:
    write_line += " " + class_name
write_line += ")\n"
write_line += "  (:predicates\n"

predicates = []
line_count = 0
for class_index, para_bind_per_class in enumerate(para_bind_overall_fault_removed):
    for fsm_no, para_bind_per_fsm in enumerate(para_bind_per_class):
        for state_index, state in enumerate(state_machines_overall_list[class_index][fsm_no]):
            predicate = ""
            write_line += "    (" + class_names[class_index] + "_fsm" + str(fsm_no) + "_state" + str(state_index)
            predicate += "    (" + class_names[class_index] + "_fsm" + str(fsm_no) + "_state" + str(state_index)
            for para_bind in para_bind_per_fsm:
                if para_bind[0][0] <= state_dict_overall[class_index][fsm_no][state]:
                    write_line += " ?"+para_bind[1] + " - " + str(class_names[para_bind[0][8]])
                    predicate += " ?"+para_bind[1] + " - " + str(class_names[para_bind[0][8]])
            if (line_count % 4) == 0:
                write_line += ")\n"
            else:
                write_line += ")"
            line_count+=1
            predicate += ")"
            predicates.append(predicate)
write_line += "  )\n"
            
for action_index, action in enumerate(actions):
    write_line += "  (:action"
    write_line += "  " + action + " "
    write_line += "  :parameters"
    write_line += "  ("
    arg_already_written_flag = False
    params_per_action = []
    args_per_action = []
    for seq in sequences:
        for actarg_tuple in seq:
            if not arg_already_written_flag:
                if actarg_tuple[0] == action:
                    arglist = []
                    for arg in actarg_tuple[1]:
                        write_line += "?"+arg + " - " + class_names[get_class_index(arg,classes)] + " "
                        arglist.append(arg)
                    args_per_action.append(arglist)
                    params_per_action.append(actarg_tuple[1])
                    arg_already_written_flag = True
    write_line += ")\n"


    # need to use finite STATE machines to get preconditions and effects.
    # Start-state = precondition. End state= Effect
    preconditions = []
    effects = []
    for arglist in params_per_action:
        for arg in arglist:
            current_class_index = get_class_index(arg, classes)
            for fsm_no, fsm in enumerate(state_machines_overall_list[current_class_index]):
                # print_table(fsm)
                df = fsm

                for i in range(df.shape[0]):
                    for j in range(df.shape[1]):
                        if df.iloc[i, j] > 0:
                            # print("(" + df.index[i] + "," + df.columns[j] + ")")
                            start_state = state_dict_overall[current_class_index][fsm_no][df.index[i]]
                            end_state = state_dict_overall[current_class_index][fsm_no][df.columns[j]]

                            start_state_index, end_state_index = -1, -1
                            for k,v in state_dict_overall[current_class_index][fsm_no].items():
                                if v == start_state:
                                    start_state_index = k
                                if v == end_state:
                                    end_state_index = k

                            for predicate in predicates:
                                pred = predicate.split()[0].lstrip('(').rstrip(')')
                                if pred == class_names[current_class_index]+"_fsm"+str(fsm_no)+"_"+str(start_state_index):

                                    if predicate not in preconditions:
                                        preconditions.append(predicate)
                                if pred == class_names[current_class_index]+"_fsm"+str(fsm_no)+"_"+str(end_state_index):
                                    if predicate not in effects:
                                        effects.append(predicate)




#     print(preconditions)
#     print(effects)
    write_line += "   :precondition"
    write_line += "   (and\n"
    for precondition in preconditions:
        # precondition = precondition.replace(?)
        write_line += "    "+precondition+"\n"
    write_line += "   )\n"
    write_line += "   :effect"
    write_line += "   (and\n"
    for effect in effects:
        write_line += "    " + effect + "\n"
    write_line += "  )"

    write_line += ")\n"

write_line += ")\n" #domain ending bracket


print(write_line)

write_file.write(write_line)
write_file.close()

;;********************Learned PDDL domain******************
(define  (domain driverlog)
  (:requirements :typing)
  (:types driver truck package location)
  (:predicates
    (driver_fsm0_state0)
    (driver_fsm0_state1)    (truck_fsm0_state0)    (truck_fsm0_state1)    (truck_fsm0_state2)
    (truck_fsm0_state3)    (truck_fsm0_state4)    (truck_fsm1_state0)    (truck_fsm1_state1)
    (truck_fsm1_state2)    (truck_fsm1_state3)    (truck_fsm2_state0)    (truck_fsm2_state1)
    (truck_fsm2_state2)    (truck_fsm2_state3)    (truck_fsm2_state4)    (truck_fsm3_state0)
    (truck_fsm3_state1)    (package_fsm0_state0)    (package_fsm0_state1)    (package_fsm0_state2)
    (location_fsm0_state0)    (location_fsm0_state1)    (location_fsm0_state2)    (location_fsm0_state3)
    (location_fsm1_state0)    (location_fsm1_state1)    (location_fsm1_state2)    (location_fsm2_state0)
    (location_fsm2_state1)    (location_fsm2_state2)    (location_fsm2_state3)    (location_fsm2_state4)
    (location_fsm3

In [85]:
# state dictionary
print_state_dictionary(state_dict_overall)

driver_fsm0_state0:['end(drivetruck.3)', 'start(disembarktruck.0)', 'end(boardtruck.0)', 'start(drivetruck.3)']
driver_fsm0_state1:['start(boardtruck.0)', 'end(disembarktruck.0)']
truck_fsm0_state0:['end(disembarktruck.1)']
truck_fsm0_state1:['end(unloadtruck.1)', 'start(disembarktruck.1)']
truck_fsm0_state2:['start(unloadtruck.1)']
truck_fsm0_state3:['end(loadtruck.1)']
truck_fsm0_state4:['start(loadtruck.1)']
truck_fsm1_state0:['start(disembarktruck.1)']
truck_fsm1_state1:['start(boardtruck.1)', 'end(disembarktruck.1)']
truck_fsm1_state2:['start(loadtruck.1)', 'end(boardtruck.1)']
truck_fsm1_state3:['end(loadtruck.1)']
truck_fsm2_state0:['start(boardtruck.1)']
truck_fsm2_state1:['end(unloadtruck.1)']
truck_fsm2_state2:['start(loadtruck.1)', 'end(boardtruck.1)']
truck_fsm2_state3:['start(unloadtruck.1)']
truck_fsm2_state4:['end(loadtruck.1)']
truck_fsm3_state0:['end(loadtruck.1)', 'start(disembarktruck.1)', 'end(unloadtruck.1)', 'end(drivetruck.0)', 'start(drivetruck.0)', 'start(unloa

# NER 

In [86]:
# finding entities using spacy
import spacy
from spacy import displacy
import en_core_web_sm
nlp = spacy.load('en_core_web_sm')
doc = nlp(coref_resolved_instructions)

In [87]:
displacy.render(nlp(str(doc)), jupyter=True, style='ent', options = {'ents':['QUANTITY', 'TIME', 'LOC', 'DATE']})